객체지향프로그래밍, file IO, 예외처리, 실행시간, 정규식, matplotlib, numpy, pandas등 library를 이용하여 구현

투수와 타자 정보 데이터 작성

[파일 내 형식]
- 투수 (한 줄 당)
    이름 구단 {구종 : 구사율} ERA AVG BB WHIP
    구단 #(선수) // 이 밑으로는 선수 인원수에 맞게 줄이 쭉 있는데 각 줄에 선수에 대한 정보
    
- 타자

  

입력 :
1. 투수의 이름,구단 입력 -> '이닝 시작' 출력 후 투수의 구종별 구사율 출력
2. 타자의 정보 입력 -> 타자의 타율, 출루율, 변화구/직구 선택률 등 정보 출력 -> 투수의 첫 구종 예상
3. trading start 입력 시 -> '1. 이적\n2. 방출' 출력 후 선택 요구
       1: 이적할 선수의 포지션, 이름, 등번호, 연봉 입력 요구 -> 이름은 다른데 등 번호가 겹칠 경우 '이미 등록된 번호입니다.' -> 이름과 등번호가 겹칠 경우 '이미 등록된 선수입니다.' 출력 ->'등록을 완료했습니다.' 출력
       2: 방출할 선수의 포지션, 이름, 등번호 -> 있다면 '정말 방출하시겠습니까?' 출력 후 y 입력 시 방출

In [20]:
import pandas as pd
def get_data_from_sheet(position, name, team):
        df = pd.read_excel(position + '.xlsx', engine='openpyxl')
        filtered_df = df[(df['Name'] == name) & (df['Team'] == team)]
        
        list_data = filtered_df.iloc[0].tolist() #선수 정보 리스트로 출력하는 코드
        return list_data

print(get_data_from_sheet('hitter', '최현우', '기아'))

['최현우', '기아', 0.342, 65, 31, 33, 0.94, 190, 225]


In [2]:
#셀에 들어 있는 사전 형식을 읽고 사전 형식으로 만드는 코드

import pandas as pd
import ast

df = pd.read_excel('pitcher.xlsx')
dict_str = df.loc[0, '구종구사율']
pitch_usage = ast.literal_eval(dict_str)
print(pitch_usage)
print(type(pitch_usage))

{'직구': 52.3, '슬라이더': 27.1, '커브': 20.6}
<class 'dict'>


In [15]:
from tabulate import tabulate
import pandas as pd
import ast


class Player:
    def __init__(self):
        self.pitcher = None
        self.fielder = None

#메뉴 출력 class
class ShowMenu(Player, Pitcher, Fielder):
    @staticmethod
    def main_menu():
        print('\033[1m[MENU]\033[0m\n1. 이닝 시작\n2. 트레이드')
        while True:
            try:
                user_input = int(input('번호를 입력하세요: '))
                if user_input == 1:
                    ShowMenu.after_pitcher_menu()
                    break
                else:
                    print('메뉴에서 선택해주세요\n')
            except ValueError:
                print('메뉴의 숫자를 입력해야 합니다\n')

    @staticmethod
    def after_pitcher_menu():
        Player.pitcher = Pitcher()
        data = [
            ['투수', '타자'],
            [Player.pitcher.info[0], '-']
        ]
        print('\n\033[1m[현재 저장된 투수/타자]\033[0m')
        print(tabulate(data, tablefmt='pretty'))
        print('\n\033[1m[MENU]\033[0m\n1. 투수 정보 출력\n2. 타자 정보 입력\n3. 투수 교체\n4. 프로그램 종료')
        
        while True:
            try:
                user_input = int(input('번호를 입력하세요: '))
                if user_input == 1:
                    print(Player.pitcher.info)
                elif user_input == 2:
                    ShowMenu.after_fielder_menu()
                    break
                elif user_input == 3:
                    Player.pitcher = Pitcher()
                elif user_input == 4:
                    print('\n프로그램을 종료합니다!')
                    break
                else:
                    print('메뉴에서 선택해주세요\n')
            except ValueError:
                print('메뉴의 숫자를 입력해야 합니다\n')

    @staticmethod
    def after_fielder_menu():
        Player.fielder = Fielder()
        data = [
            ['투수', '타자'],
            [Player.pitcher.info[0], Player.fielder.info[0]]
        ]
        print('\n\033[1m[현재 저장된 투수/타자]\033[0m')
        print(tabulate(data, tablefmt='pretty'))
        print('\n\033[1m[MENU]\033[0m\n1. 투수 정보 출력\n2. 타자 정보 출력\n3. 새로운 타자 정보 입력\n4. 투수 교체\n5. 프로그램 종료')
        while True:
            try:
                user_input = int(input('번호를 입력하세요: '))
                if user_input == 1:
                    print(Player.pitcher.info)
                elif user_input == 2:
                    print(Player.fielder.info)
                elif user_input == 3:
                    Player.fielder = Fielder()
                elif user_input == 4:
                    Player.pitcher = Pitcher()
                elif user_input == 5:
                    print('\n프로그램을 종료합니다!')
                    break
                else:
                    print('메뉴에서 선택해주세요\n')
            except ValueError:
                print('메뉴의 숫자를 입력해야 합니다\n')

    
class GetList:
    @staticmethod
    def get_data_from_sheet(position, name, team):
        df = pd.read_excel(position + '.xlsx', engine='openpyxl')
        filtered_df = df[(df['Name'] == name) & (df['Team'] == team)]
        
        list_data = filtered_df.iloc[0].tolist()
        return list_data


        
class Pitcher(GetList):
    def __init__(self):
        print('\n\033[1m[투수 정보 입력]\033[0m')
        self.name = input('투수 이름: ')
        self.team = input('투수 구단: ')
        self.info = super().get_data_from_sheet('pitcher', self.name, self.team)

class Fielder(GetList):
    def __init__(self):
        print('\n\033[1m[야수 정보 입력]\033[0m')
        self.name = input('야수 이름: ')
        self.team = input('야수 구단: ')
        self.info = super().get_data_from_sheet('fielder', self.name, self.team)



#투수와 야수 정보 출력 class
class ShowInfo(Fielder, Pitcher):
    def show_fielder(self):
        print(f'[{Fielder.name}의 정보입니다.]')
        print(Fielder.info)
        
    def show_pitcher(self):
        print(f'[{Fielder.name}의 정보입니다.]')
        print(Fielder.info)

if __name__ == "__main__":
    player = Player()
    ShowMenu.main_menu()
    #else 로 2를 입력받을 시 trade class 구현
        

[MENU]
1. 이닝 시작
2. 트레이드


번호를 입력하세요:  1



[투수 정보 입력]


투수 이름:  폰세
투수 구단:  한화



[현재 저장된 투수/타자]
+------+------+
| 투수 | 타자 |
| 폰세 |  -   |
+------+------+

[MENU]
1. 투수 정보 출력
2. 타자 정보 입력
3. 투수 교체
4. 프로그램 종료


...


[현재 저장된 투수/타자]
+------+--------+
| 투수 |  타자  |
| 폰세 | 김성윤 |
+------+--------+

[MENU]
1. 투수 정보 출력
2. 타자 정보 출력
3. 새로운 타자 정보 입력
4. 투수 교체
5. 프로그램 종료


번호를 입력하세요:  2


['김성윤', '삼성', np.float64(0.358), np.int64(64), np.int64(25), np.int64(28), np.float64(0.89), np.int64(179), np.int64(212)]


번호를 입력하세요:  5



프로그램을 종료합니다!


In [8]:
# 굵은 텍스트 출력
print("\033[1m굵은 텍스트입니다\033[0m")


굵은 텍스트입니다
